- 因為查詢職缺最多有20\*150=3000筆資料，所以要將查詢的的條件透過條件進行篩選、排列組合，藉此撈出完整的資料
- 資料分為 列表區 與 詳細內容區，要分兩次取資料
- 有時候request過快會連線失敗，在這裡用while迴圈來進行處理
- 因為資料量大，建議按照組別存一次資料，這樣遇到錯誤時才不用重新爬資料
- 由公司資訊：地址，聯絡電話、職務

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import time
import os
headers={'useer-agents':'GoogleBot'}

In [2]:
# 地區代碼
url = 'https://static.104.com.tw/category-tool/json/Area.json'
resp = requests.get(url)
df1 = []
for i in resp.json()[0]['n']:
    ndf = pd.DataFrame(i['n'])
    ndf['city'] = i['des']
    df1.append(ndf)
df1=pd.concat(df1, ignore_index=True)
df1 = df1.loc[:,['city','des','no']]
df1 = df1.sort_values('no')
df1

,city,des,no
0,台北市,台北市中正區,6001001001
1,台北市,台北市大同區,6001001002
2,台北市,台北市中山區,6001001003
3,台北市,台北市松山區,6001001004
4,台北市,台北市大安區,6001001005
...,...,...,...
360,金門縣,金門縣烏坵鄉,6001022006
361,連江縣,連江縣南竿鄉,6001023001
362,連江縣,連江縣北竿鄉,6001023002
363,連江縣,連江縣莒光鄉,6001023003


In [3]:
url= 'https://static.104.com.tw/category-tool/json/JobCat.json'
resp = requests.get(url)
df2 = []
for i in resp.json():
    for j in i['n']:
        ndf = pd.DataFrame(j['n'])
        ndf['des1'] = i['des']# 職務大分類
        ndf['des2'] = j['des']# 職務小分類
        df2.append(ndf)
df2 = pd.concat(df2, ignore_index=True)
df2 = df2.loc[:,['des1', 'des2', 'des', 'no']]
df2 = df2.sort_values('no')
df2

,des1,des2,des,no
1,經營╱人資類,經營╱幕僚類人員,經營管理主管,2001001001
0,經營╱人資類,經營╱幕僚類人員,儲備幹部,2001001002
2,經營╱人資類,經營╱幕僚類人員,主管特別助理,2001001003
3,經營╱人資類,人力資源類人員,人力資源主管,2001002001
6,經營╱人資類,人力資源類人員,人力資源人員,2001002002
...,...,...,...,...
461,其他職類,其他類人員,加油員,2018002010
462,其他職類,其他類人員,派報生╱傳單派送,2018002011
463,其他職類,其他類人員,清潔工╱資源回收人員,2018002012
464,其他職類,其他類人員,家事服務人員,2018002013


In [4]:
tmp = pd.DataFrame([re.sub('\.pkl','',file)for file in os.listdir('./data')],columns=['no'])
df1 = pd.merge(df1, tmp, how='left',on='no',indicator=True)
df1 = df1.loc[df1['_merge']!='both',:]
df1

,city,des,no,_merge
108,台中市,台中市西區,6001008004,left_only
109,台中市,台中市北區,6001008005,left_only
110,台中市,台中市北屯區,6001008006,left_only
111,台中市,台中市西屯區,6001008007,left_only
112,台中市,台中市南屯區,6001008008,left_only
...,...,...,...,...
360,金門縣,金門縣烏坵鄉,6001022006,left_only
361,連江縣,連江縣南竿鄉,6001023001,left_only
362,連江縣,連江縣北竿鄉,6001023002,left_only
363,連江縣,連江縣莒光鄉,6001023003,left_only


In [ ]:
columns = ['公司名稱','公司編號','公司類別','公司類別描述', '公司連結','職缺名稱','職缺編號','更新日期','職缺連結','公司地址','地區','經歷','學歷']

for areades, areacode in zip(df1['des'],df1['no']):
    values = []
    for jobdes, jobcode in zip(df2['des'],df2['no']):
        print(areades, areacode, jobdes, jobcode)
        page = 1
        while page <150:
            try:
                url = 'https://www.104.com.tw/jobs/search/?ro=0&jobcat={}&jobcatExpansionType=1&area={}&order=11&asc=0&page={}&mode=s&jobsource=2018indexpoc'.format(jobcode, areacode, page)
                print(url)
                resp = requests.get(url,headers=headers)
                soup = BeautifulSoup(resp.text)
                soup2 = soup.find('div',{'id':'js-job-content'}).findAll('article',{'class':'b-block--top-bord job-list-item b-clearfix js-job-item'})
                print(len(soup2))

                for job in soup2:
                                        
                    update_date = job.find('span',{'class':'b-tit__date'}).text
                    update_date = re.sub('\r|\n| ','',update_date)

                    try:
                        address = job.select('ul > li > a')[0]['title']
                        address = re.findall('公司住址：(.*?)$',address)[0]
                    except:
                        address = ''
                   
                    loc = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[0].text
                    exp = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[1].text
                    try:
                        edu = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[2].text
                    except:
                        edu = ''
                    value = [job['data-cust-name'], # 公司名稱
                             job['data-cust-no'], # 公司編號
                             job['data-indcat'], # 公司類別
                             job['data-indcat-desc'], # 公司類別描述
                             job.select('ul > li > a')[0]['href'], # 公司連結
                             job['data-job-name'],# 職缺名稱
                             job['data-job-no'],# 職缺編號
                             update_date, # 更新日期
                             job.find('a',{'class':'js-job-link'})['href'], # 職缺連結
                             address,# 公司地址
                             loc, # 地區
                             exp,# 經歷
                             edu  # 學歷
                            ]
                    values.append(value)
                
                page+=1
                print(len(values))
                if len(soup2) < 20:
                    break
            except:
                print('Retry')
    df = pd.DataFrame()
    df = pd.DataFrame(values, columns=columns)
    df.to_pickle('./data/' + areacode + '.pkl')
    print('===================================  Save Data  ===================================')

台中市西區 6001008004 經營管理主管 2001001001
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001001001&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
20
20
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001001001&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=2&mode=s&jobsource=2018indexpoc
20
40
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001001001&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=3&mode=s&jobsource=2018indexpoc
10
50
台中市西區 6001008004 儲備幹部 2001001002
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001001002&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
20
70
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001001002&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=2&mode=s&jobsource=2018indexpoc
20
90
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2001001002&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=3&mode=s&jobsource=2018indexpoc
20
110
h

20
777
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2002001012&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=4&mode=s&jobsource=2018indexpoc
20
797
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2002001012&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=5&mode=s&jobsource=2018indexpoc
4
801
台中市西區 6001008004 法務╱智財主管 2002002001
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2002002001&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
4
805
台中市西區 6001008004 律師 2002002002
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2002002002&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
12
817
台中市西區 6001008004 代書╱地政士 2002002003
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2002002003&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
6
823
台中市西區 6001008004 法務人員 2002002004
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2002002004&jobcatExpansionType=1&

20
1363
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2003002011&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=2&mode=s&jobsource=2018indexpoc
8
1371
台中市西區 6001008004 催收人員 2003002012
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2003002012&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
4
1375
台中市西區 6001008004 核保╱保險內勤人員 2003002013
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2003002013&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
5
1380
台中市西區 6001008004 理賠人員 2003002014
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2003002014&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
2
1382
台中市西區 6001008004 券商後線人員 2003002015
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2003002015&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
5
1387
台中市西區 6001008004 股務人員 2003002016
https://www.104.com.tw/jobs/search/?ro=0

13
1998
台中市西區 6001008004 營運管理師╱系統整合╱ERP專案師 2004003002
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2004003002&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
6
2004
台中市西區 6001008004 軟體相關專案管理師 2004003003
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2004003003&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
3
2007
台中市西區 6001008004 其他專案管理師 2004003004
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2004003004&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
13
2020
台中市西區 6001008004 產品管理師 2004003005
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2004003005&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
9
2029
台中市西區 6001008004 客戶服務主管 2005001001
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2005001001&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
Retry
https://www.104.com.tw/

20
2799
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2005003001&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=2&mode=s&jobsource=2018indexpoc
20
2819
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2005003001&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=3&mode=s&jobsource=2018indexpoc
20
2839
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2005003001&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=4&mode=s&jobsource=2018indexpoc
20
2859
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2005003001&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=5&mode=s&jobsource=2018indexpoc
20
2879
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2005003001&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=6&mode=s&jobsource=2018indexpoc
6
2885
台中市西區 6001008004 國外業務主管 2005003002
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2005003002&jobcatExpansionType=1&area=6001008004&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
6
2891
台中市西區 6001008004 

...

20
3695
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2006001001&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=7&mode=s&jobsource=2018indexpoc
20
3715
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2006001001&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=8&mode=s&jobsource=2018indexpoc
20
3735
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2006001001&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=9&mode=s&jobsource=2018indexpoc
20
3755
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2006001001&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=10&mode=s&jobsource=2018indexpoc
20
3775
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2006001001&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=11&mode=s&jobsource=2018indexpoc
20
3795
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2006001001&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=12&mode=s&jobsource=2018indexpoc
13
3808
台中市北屯區 6001008006 中餐廚師 2006001002
https://www.1

6
4273
台中市北屯區 6001008006 美甲彩繪師 2006003007
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2006003007&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
7
4280
台中市北屯區 6001008006 美容類助理 2006003008
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2006003008&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
14
4294
台中市北屯區 6001008006 美髮類助理 2006003009
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2006003009&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
7
4301
台中市北屯區 6001008006 美睫師 2006003010
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2006003010&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
0
4301
台中市北屯區 6001008006 軟體專案主管 2007001001
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2007001001&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
6
4307
台中市北屯區 6001008006 電子商務技術主管 2007001002


0
4618
台中市北屯區 6001008006 數位IC設計工程師 2008001015
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2008001015&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
1
4619
台中市北屯區 6001008006 半導體工程師 2008001016
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2008001016&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
0
4619
台中市北屯區 6001008006 微機電工程師 2008001017
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2008001017&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
Retry
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2008001017&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
0
4619
台中市北屯區 6001008006 光電工程師 2008001018
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2008001018&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
0
4619
台中市北屯區 6001008006 光學工程師 2008001019
https://www.104.com.tw/jobs/searc

5
4673
台中市北屯區 6001008006 品管╱品保工程師 2009003002
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2009003002&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
10
4683
台中市北屯區 6001008006 可靠度工程師 2009003003
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2009003003&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
0
4683
台中市北屯區 6001008006 硬體測試工程師 2009003004
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2009003004&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
0
4683
台中市北屯區 6001008006 測試人員 2009003005
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2009003005&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
0
4683
台中市北屯區 6001008006 IC封裝╱測試工程師 2009003006
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2009003006&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
0
4683
台中市北屯區 6001008006 軟韌體測試工程師 

1
4781
台中市北屯區 6001008006 推土機及有關設備操作員 2010001026
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2010001026&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
2
4783
台中市北屯區 6001008006 吊車╱起重機及有關設備操作員 2010001027
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2010001027&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
2
4785
台中市北屯區 6001008006 手工包裝工及有關工作者 2010001028
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2010001028&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
4
4789
台中市北屯區 6001008006 其他機械操作員 2010001029
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2010001029&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
4
4793
台中市北屯區 6001008006 鑄造╱鍛造模具技術人員 2010001030
https://www.104.com.tw/jobs/search/?ro=0&jobcat=2010001030&jobcatExpansionType=1&area=6001008006&order=11&asc=0&page=1&mode=s&jobsource=2018indexpoc
0
4793
台中市北屯區 60

In [4]:
url = 'https://www.104.com.tw/jobs/search/?ro=0&jobcat=2004001006&jobcatExpansionType=1&area=6001006001&order=11&asc=0&page=2&mode=s&jobsource=2018indexpoc'
resp = requests.get(url,headers=headers)
soup = BeautifulSoup(resp.text)
soup2 = soup.find('div',{'id':'js-job-content'}).findAll('article',{'class':'b-block--top-bord job-list-item b-clearfix js-job-item'})
soup2


[<article class="b-block--top-bord job-list-item b-clearfix js-job-item" data-cust-name="(總公司)南山人壽保險股份有限公司" data-cust-no="11456006000" data-indcat="1004003001" data-indcat-desc="人身保險業" data-is-apply="0" data-is-save="0" data-job-name="[新鮮人專區]業務活動推廣人員(內勤/新竹)" data-job-no="10317456" data-job-ro="1" data-jobsource="2018indexpoc">
 <div class="b-block__left">
 <h2 class="b-tit">
 <span class="b-tit__date">
                                             6/12
                                     </span>
 <a class="js-job-link" href="//www.104.com.tw/job/65500?jobsource=2018indexpoc" target="_blank">[新鮮人專區]業務活動推廣人員(內勤/新竹)</a>
 </h2>
 <ul class="b-list-inline b-clearfix">
 <li class="b-list-inline__ad-icon">
 <svg class="b-icon--gray b-icon--w16">
 <use xlink:href="//www.104.com.tw/jobs/search/static/img/sprite.svg#icon-company"></use>
 </svg>
 </li>
 <li>
 <a href="//www.104.com.tw/company/59glysw?jobsource=2018indexpoc" target="_blank" title="公司名：(總公司)南山人壽保險股份有限公司
 公司住址：新竹市">
                 

In [5]:
for i, job in enumerate(soup2):
    print('='*44, i, '='*44)
    print(job['data-cust-name'])
    print(job['data-cust-no'])
    print(job['data-indcat'])
    print(job['data-indcat-desc'])
    print(job['data-job-name'])
    print(job['data-job-no'])
    print(job.find('a',{'class':'js-job-link'})['href'])
    print(job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[0].text)
    print(job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[1].text)
    print(job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[2].text)
    print(job.select('ul > li > a')[0]['href'])


============================================ 0 ============================================
(總公司)南山人壽保險股份有限公司
11456006000
1004003001
人身保險業
[新鮮人專區]業務活動推廣人員(內勤/新竹)
10317456
//www.104.com.tw/job/65500?jobsource=2018indexpoc
新竹市
經歷不拘
大學
//www.104.com.tw/company/59glysw?jobsource=2018indexpoc
============================================ 1 ============================================
大魯閣實業股份有限公司
4690983000
1003002013
百貨相關業
【大魯閣湳雅廣場】行銷企劃專員
11679350
//www.104.com.tw/job/6ybue?jobsource=2018indexpoc
新竹市
2年以上
專科
//www.104.com.tw/company/25kw1nc?jobsource=2018indexpoc
============================================ 2 ============================================
財團法人農業科技研究院
130000000034229
1013002002
職業團體
研究專員-產業分析組(I)
11631737
//www.104.com.tw/job/6xb3t?jobsource=2018indexpoc
新竹市
經歷不拘
碩士
//www.104.com.tw/company/1a2x6bippx?jobsource=2018indexpoc
============================================ 3 ============================================
名豐電子股份有限公司
53179074000
1001001002
電腦軟體服務業
行銷企劃專員
11381861
//www.

IndexError: list index out of range

In [16]:
soup2[16].select('ul > li > a')

[<a href="//case.104.com.tw" target="_blank" title="公司名：104外包網">
                         104外包網
                     </a>]

In [15]:
job.select('ul > li')[4]

IndexError: list index out of range